## TAREA: Imputación de missing values con modelo predictivo

### Tras estudiar las estrategias de tratamiento de valores numéricos no disponibles como se explica en el notebook de preprocesado del dataset housing, propón una nueva solución para utilizando la clase KNNImputer de scikit-learn.

In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer # Importamos la clase KNNImputer
from sklearn.model_selection import train_test_split

housing = pd.read_csv("./data/housing.csv") 

# Generación de conjuntos de entrenamiento y prueba mediante muestreo estratificado por ingreso medio
train_set, test_set = train_test_split(housing, test_size=0.2,
    stratify=pd.cut(housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5]),
    random_state=42
    )

housing = train_set.drop("median_house_value", axis=1) # Eliminamos la columna de la variable dependiente
housing_labels = train_set["median_house_value"].copy() # Guardamos la variable dependiente (etiquetas)

# Analizamos los campos con valores nulos de nuestro conjunto de datos
print("Sumatorio de nulos del dataset original\n")
print(housing.isna().sum()) 

# índices de las filas con valores nulos
null_rows_idx = housing.isnull().any(axis=1) 

# mostramos las 5 primeras filas con nulo
print("\nFilas con nulo\n")
print(housing.loc[null_rows_idx].head())

# podemos seleccionar las columnas numéricas para desechar la columna de tipo objeto o borrar directamente la columna en cuestión
#housing= housing.select_dtypes(include=[np.number]) 
housing = housing.drop(columns="ocean_proximity", axis=1)

# construimos el objeto KNNImputer, pasándole el nº de vecinos más cercanos que va a utilizar para el algoritmo
# este algoritmo imputa los valores faltantes utilizando la media ponderada o no ponderada del número deseado de vecinos más cercanos
# si modificamos el nº de vecinos, podemos comprobar cómo van cambiando los valores que se asignan a los nulos (total_bedroom en nuestro caso)
imputer = KNNImputer(missing_values=np.NaN, n_neighbors=10) 
# empleamos el algoritmo de imputación a valores nulos
housing_num_array_tr = imputer.fit_transform(housing)

housing_df = pd.DataFrame(housing_num_array_tr,
                            columns=housing.columns,
                            index= housing.index)

#vemos que ya no tenemos valores nulos en nuestro campo total_bedrooms
print("\n\nSumatorio de nulos del dataset tras aplicar el KNN\n")
print(housing_df.isna().sum())

print("\n\nFilas que tenían nulos tras aplicar el KNN\n")
print(housing_df.loc[null_rows_idx].head())


Sumatorio de nulos del dataset original

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        158
population              0
households              0
median_income           0
ocean_proximity         0
dtype: int64

Filas con nulo

       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
1606     -122.08     37.88                26.0       2947.0             NaN   
10915    -117.87     33.73                45.0       2264.0             NaN   
19150    -122.70     38.35                14.0       2313.0             NaN   
4186     -118.23     34.13                48.0       1308.0             NaN   
16885    -122.40     37.58                26.0       3281.0             NaN   

       population  households  median_income ocean_proximity  
1606        825.0       626.0         2.9330        NEAR BAY  
10915      1970.0       499.0         3.4193       <1H OCEAN  
19150       954.0       397.0  